## 多层网络的实践案例

### 基础库
这里需要导入的库有：

- numpy：这个不用介绍
- h5py 用来分析H5 文件的包.
- matplotlib 不用介绍
- PIL and scipy 用来分析最后的预测结果.
- dnn_utils：作者给定的一些必要的函数：sigmoid, sigmoid_backward, relu, relu_backward
- testCases 测试函数
- np.random.seed(1) 保证随机数的确定性

In [1]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from dnn_app_utils_v2 import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

### 数据集

这里的问题就是想通过一个模型来判别给出的图片是否为猫。
- 图片的shape(width,hight,3),RGB的图片
- 这里的图片数据是保存在一个“data.h5”的文件中

现在开始来获取训练集和测试集。就是利用上面的给的脚本函数：
- train_set_x_orig ： 训练集orig,表示原始数据，后面我们要对原始数据进行归一化等处理
- traing_set_y：训练集标签
- test_set_x_orig：测试集orig,表示原始数据
- test_set_y：测试集标签
- classes：种类

这些数据的格式通过shape函数就可以获得：
- train_set_x_orig：(209, 64, 64, 3)
    - 209：表示训练样本的个数
    - (64, 64, 3):表示图片size 和channel
- traing_set_y：(1, 209)
- test_set_x_orig：(50, 64, 64, 3)
- test_set_y：(1, 50)
- classes：(2,)
    - non-cat
    - cat

In [2]:
#获取数据集
train_set_x_orig,train_set_y,test_set_x_orig,test_set_y, classes= load_data()

# 变换数据
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

# 这里的.T表示转置，为了与后面的W相匹配，也方便大家理解输入的神经元
train_set_x_flat = train_set_x_orig.reshape(m_train,-1).T
test_set_x_flat = test_set_x_orig.reshape(m_test,-1).T

print(train_set_x_flat.shape)
print(test_set_x_flat.shape)

#### 数据的归一化
train_x = train_set_x_flat/255.
test_x = test_set_x_flat/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

(12288, 209)
(12288, 50)
train_x's shape: (12288, 209)
test_x's shape: (12288, 50)


### L层网络的模型结构

其结构图如下：
![](imgs/19.jpg)

- 输入时(64,64,3)的图片，拉伸转换成一个列向量(12288,1)
- 这里的第一层网络隐层层为n1,那么权重W1 shape为(n1,12288)
- 经过第一层的relu激活函数后，A1就变成(n1,1)
- 循环执行上面两步创建更多的层，
- 最后经过一层sigmoid激活函数，输出该值。

### 创建模型的一般步骤

- 初始化参数并定义超参数
- 循环更新：
    - 前向传播
    - 计算损失值
    - 反向传播
    - 更新参数
- 使用训练的数据预测

### 多层神经网络的创建

其结果的流程如下：
[线性-> Relu激活]x(L-1) -> 线性 -> Sigmoid 激活
需要使用上一节定义的一些基础函数如下：

In [3]:
def initialize_parameters(n_x, n_h, n_y):
    """
    输入：
        n_x: 表示输入的数据的个数
        n_h: 中间隐藏层的神经元的个数
        n_y: 表示输出的数据的个数
    """
    
    np.random.seed(1)
    # W1的shape是根据前一层和后一层的维度确定
    W1 = np.random.randn(n_h,n_x) * 0.01
    # b的维度是根据当前层的维度来确定
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    
    # 辅助判断参数的shape
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters={"W1":W1,
               "b1":b1,
               "W2":W2,
               "b2":b2}
    
    return parameters

def initialize_parameters_deep(layer_dims):
    """
    输入：
        layer_dims：是各个层的神经元的个数组成的数组
    """
    np.random.seed(3)
    parameters = {}
    # 获得数组的程度
    L = len(layer_dims)
    
    # 从第二层开始
    for l in range(1,L):
        # 为了获得前一层的维度，需要减1
        parameters['W'+str(l)] = np.random.randn(
            layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((
            layer_dims[l],1))
        
    return parameters
    


def linear_forward(A, W, b):
    """
    输入：
        A:表示前一层的计算结果
        W:表示当前层的权重
        b:表示当前层的偏置
        
    输出：
        Z: 计算结果
        cache:A,W,b的数据
    """
    Z = np.dot(W,A) + b
    
    cache = (A,W,b)
    
    return Z,cache

def linear_activation_forward(A_prev, W, b, activation):
    """
    输入：
    A_prev:表示上一层的激活函数计算的值或者输入层数据
    W,b:表述参数
    activation： 表示当前激活函数样式："sigmoid"&"relu"
    
    输出：
    A:当前层的输出
    cache:缓存linear_cache, activation_cache
    """
    
    if activation == "sigmoid":
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = relu(Z)
    
    cache = (linear_cache, activation_cache)
    return A,cache

def L_model_forward(X, parameters):
    """
    输入：
        X: 输入的数据
        parameters:具体的参数，
    """
    
    caches = []
    A = X
    # 通过参数就可以获得模型的层数
    L = len(parameters) // 2
    
    # 前L-1层的网络
    for l in range(1,L):
        A_prev = A
        A,cache = linear_activation_forward(A_prev,
                            parameters['W'+str(l)],
                            parameters['b'+str(l)],
                                           'relu')
        caches.append(cache)
        
    # 最后一层的网络   
    AL,cache = linear_activation_forward(A,
                            parameters['W'+str(L)],
                            parameters['b'+str(L)],
                                        'sigmoid')
    caches.append(cache)
    
    return AL, caches

def compute_cost(AL,Y):
    """
    输入：
        AL:最后一层的输出值
        Y:标签数据
    """
    
    m = Y.shape[1]
    # j计算loss值
    cost = -np.sum(np.multiply(np.log(AL),Y)+
                   np.multiply(np.log(1-AL),1-Y))/m
    
    cost = np.squeeze(cost)
    
    return cost


def linear_backward(dZ, cache):
    """
    输入：
        dZ 前一层的Activate导数
        cache: 该层缓存的（A_prev,W,b）
    
    输出：
        dA_prev：loss的梯度
        dW:W的梯度
        db:b的梯度
    """
    A_prev,W,b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev,dW,db


def linear_activation_backward(dA,cache,activation):
    """
    输入：
        dA: 前一层的梯度
        cache: 前向传播缓存的值：linear_cache,activation_catche
        activation: 激活函数的形式
    """
    
    linear_cache, activation_cache = cache
    
    
    if activation == 'relu':
        dZ = relu_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
        
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
        
    return dA_prev,dW,db

def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # 这里计算loss function的梯度
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # 计算最后一层linear-sigmoid的梯度
    current_cache = caches[L-1]
    grads["dA"+str(L)],grads["dW"+str(L)],grads["db"+str(L)]=linear_activation_backward(dAL, 
                                                                                        current_cache,"sigmoid")
    
    # 迭代计算其他层linear-relu的梯度
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        grads["dA"+str(l+1)],grads["dW"+str(l+1)],grads["db"+str(l+1)]=linear_activation_backward(grads["dA"+str(l+2)],
                                                                                            current_cache,"relu")
             
    return grads
    


def update_parameters(parameters, grads, learning_rate):
    """
    输入：
        parameters：缓存的参数字典
            parameters['W' + str(l)] = Wl
            parameters['b' + str(l)] = bl
        grads: 缓存的梯度字典
            grads['dW' + str(l)] = dWl
            grads['db' + str(l)] = dbl
        learning_rate：学习率
    """
    L = len(parameters)//2
    
    for l in range(L): # 从零开始，而参数是从1开始W1
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [4]:
# 5层网络模型
layers_dims = [12288, 20, 7, 5, 1] 

In [7]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    构建一个两层神经网络模型: LINEAR->RELU->LINEAR->SIGMOID.

    输入:
    X -- 输入数据
    Y -- 标签数据 (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- 网络的结构维度 (n_x, n_h, n_y)
    num_iterations -- 循环优化的次数
    learning_rate -- 学习率
    print_cost -- 是否需要每隔100次打印cost值 

    输出:
    parameters -- 字典 W1, W2, b1, and b2
    """
    
    
    # 基础参数
    np.random.seed(1)
    costs = []

    
    parameters = initialize_parameters_deep(layers_dims)
    
    # 开始迭代更新
    for i in range(0,num_iterations):
        
        AL,caches = L_model_forward(X, parameters)
        
        # 计算loss值
        cost = compute_cost(AL,Y)
        
        grads = L_model_backward(AL, Y, caches)
        
        parameters = update_parameters(parameters, 
                                       grads,
                                       learning_rate)
        
        # 打印loss 值

        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)

    # 绘制cost图
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    return parameters
    
    
    
   

In [8]:
parameters = L_layer_model(train_x, train_set_y,
                             layers_dims,
                             num_iterations = 2500, 
                             print_cost=True)

Cost after iteration 0: 0.693148
Cost after iteration 100: 0.678011
Cost after iteration 200: 0.667600
Cost after iteration 300: 0.660422
Cost after iteration 400: 0.655458
Cost after iteration 500: 0.652013
Cost after iteration 600: 0.649616
Cost after iteration 700: 0.647942


KeyboardInterrupt: 